In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../../")
print(os.getcwd())

C:\Users\Milosz\Desktop\python\thesis-recsys


In [2]:
import pandas as pd
import numpy as np
import functools
import operator
import json
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
games = pd.read_csv('data/games.csv')
rec = pd.read_csv('data/recommendations.csv')
users = pd.read_csv('data/users.csv')
meta = pd.read_json("data/games_metadata.json", lines=True)

In [4]:
meta

,app_id,description,tags
0,10090,"Call of Duty is back, redefining war like you'...","[Zombies, World War II, FPS, Multiplayer, Acti..."
1,13500,Enter the dark underworld of Prince of Persia ...,"[Action, Adventure, Parkour, Third Person, Gre..."
2,22364,,[Action]
3,113020,Monaco: What's Yours Is Mine is a single playe...,"[Co-op, Stealth, Indie, Heist, Local Co-Op, St..."
4,226560,Escape Dead Island is a Survival-Mystery adven...,"[Zombies, Adventure, Survival, Action, Third P..."
...,...,...,...
48348,248410,The planet of Aethereus is an ancient and myst...,"[RPG, Action, Indie, Multiplayer, Co-op, Kicks..."
48349,839680,"No, Pay to Win/ Yes, Original Military FPS OBT...","[Free to Play, FPS, PvP, War, Combat, Modern, ..."
48350,758210,Look for hints and disable the bomb. Now!! You...,"[Adventure, Early Access, Simulation, VR, Esca..."
48351,1612360,As a new Pvt on camp Greenwood you will be tes...,"[Action, Simulation, Racing, Shooter, FPS, VR,..."


In [5]:
games

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,10090,Call of Duty: World at War,2008-11-18,True,False,False,Very Positive,92,37039,19.99,19.99,0.0,True
1,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
2,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
3,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
4,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48348,248410,Legends of Aethereus,2013-09-27,True,True,True,Mostly Negative,23,142,0.00,0.00,0.0,True
48349,839680,The Front of Greed,2018-08-01,True,False,False,Mostly Negative,37,27,0.00,0.00,0.0,True
48350,758210,Escape Room,2017-12-11,True,False,False,Mostly Negative,23,30,0.00,0.00,0.0,True
48351,1612360,Pistol Master!,2021-05-20,True,False,False,Negative,18,11,0.00,0.00,0.0,True


In [6]:
rec

,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
0,975370,0,0,2022-12-12,True,36.3,21265,0
1,304390,4,0,2017-02-17,False,11.5,1166,1
2,1085660,2,0,2019-11-17,True,336.5,97101,2
3,703080,0,0,2022-09-23,True,27.4,99068,3
4,526870,0,0,2021-01-10,True,7.9,9721,4
...,...,...,...,...,...,...,...,...
11480207,311690,0,0,2020-09-17,True,16.1,3250115,11480207
11480208,1449850,0,0,2022-02-01,False,0.7,42845,11480208
11480209,960090,3,0,2020-11-26,True,200.2,190650,11480209
11480210,311690,0,0,2018-01-07,True,5.7,4429817,11480210


In [7]:
users

,user_id,products,reviews
0,6104846,156,1
1,3841247,329,3
2,2074781,176,2
3,2236937,98,2
4,2476707,144,3
...,...,...,...
6230639,2137695,3,0
6230640,2140035,0,0
6230641,2141140,0,0
6230642,2141972,0,0


In [ ]:
def plot_monthly_interactions(df: pd.DataFrame, col: str, boundary: str = None):
    """
    Plot interactions from `df[col]` aggregated per month as line chart.
    
    Args:
        df: Dataset containing `col`
        col: Name of column with dates 
        boundary: Date in format `yyyy-mm-dd` representing boundary of split
    """
    
    fig, ax = plt.subplots()
    
    month_ints = pd.to_datetime(df[col]).dt.to_period("M").value_counts().sort_index()
    #month_ints = df[col].str.slice(0, 7).value_counts().sort_index()
    month_ints.index = month_ints.index.to_timestamp()
    
    ax.plot(month_ints)
    
    if boundary:
        from datetime import datetime
        boundary = datetime.strptime("2022-12-01", "%Y-%m-%d")
        ax.axvline(boundary, color='red', linestyle='--')
        
    plt.show()

In [ ]:
plot_monthly_interactions(rec, "date", "2022-12-01")

In [ ]:
rec['is_recommended'] = rec['is_recommended'].astype(float)

In [ ]:
rec = rec[rec['is_recommended'] == 1.0]

In [ ]:
def split_by_date(df: pd.DataFrame, split_date: str, col: str):
    """
    Sort and split dataframe into train and test sets on given date threshold.
    
    Args:
        df: Dataset containing `col`
        split_date: Split date in format `yyyy-mm-dd`
        col: Name of column with dates
        
    Returns:
        Tuple of (train_set, test_set) where train contains rows before `date`
    """
    df_sorted = df.sort_values(by=col)
    mask = df_sorted[col] <= split_date
    df_train = df_sorted[mask]
    df_test = df_sorted[~mask]
    
    return df_train, df_test

In [ ]:
rec_train, rec_test = split_by_date(rec, "2022-11-01", "date")

In [ ]:
rec_train

In [ ]:
def split_transductive(df_train, df_test, user_col='user_id', item_col='app_id', print_stats=False):
    all_users, all_items = df_train[user_col].unique(), df_train[item_col].unique()
    
    df_test_trans = df_test[(df_test[user_col].isin(all_users)) & \
                            (df_test[item_col].isin(all_items))]
    
    if print_stats:
        u_test, i_test = df_test[user_col].unique().size, df_test[item_col].unique().size
        u_test_trans, i_test_trans = df_test_trans[user_col].unique().size, \
                                     df_test_trans[item_col].unique().size
        
        print(f"Train User nodes: {all_users.size} \nTrain Item nodes: {all_items.size} \n\n\
Test User nodes w/o intersection: {u_test} \nTest Item nodes w/o intersection: \
{i_test} \n\nTest User nodes: {u_test_trans} \nTest Item nodes: {i_test_trans}")
        
    return df_train, df_test_trans

In [ ]:
rec_train, rec_test = split_transductive(rec_train, rec_test, print_stats=True)

In [ ]:
# u_train, a_train = rec_train.user_id.unique(), rec_train.app_id.unique()
# u_test, a_test = rec_test.user_id.unique(), rec_test.app_id.unique()

In [ ]:
# u = np.intersect1d(u_train, u_test)
# a = np.intersect1d(a_train, a_test)

In [ ]:
# u.size, a.size

In [ ]:
#u, a = rec_train.user_id.unique(), rec_train.app_id.unique()

In [ ]:
#print(u.size, a.size)

In [ ]:
#print(rec_test.user_id.unique().size, rec_test.app_id.unique().size)

In [ ]:
#rec_test = rec_test[(rec_test.user_id.isin(u)) & (rec_test.app_id.isin(a))]

In [ ]:
#print(rec_test.user_id.unique().size, rec_test.app_id.unique().size)

In [ ]:
# rec_train = rec_train[(rec_train.user_id.isin(u)) & (rec_train.app_id.isin(a))]
# rec_test = rec_test[(rec_test.user_id.isin(u)) & (rec_test.app_id.isin(a))]

In [ ]:
#rec_train

In [ ]:
#np.setdiff1d(rec_train.user_id.unique(), rec_test.user_id.unique()).shape

In [ ]:
#np.setdiff1d(rec_train.app_id.unique(), rec_test.app_id.unique()).shape

In [ ]:
#np.setdiff1d(rec_test.app_id.unique(), rec_train.app_id.unique())

In [ ]:
for ui in np.intersect1d(u, rec_test.user_id.unique()):
    print("\n\n", ui)
    print("TRAIN: ")
    display(rec_train[rec_train.user_id == ui])
    print("TEST: ")
    display(rec_test[rec_test.user_id == ui])

In [ ]:
def remap(df, col):
    idx = df[col].unique()
    new_idx = np.arange(idx.size)
    return {i: ni for i, ni in zip(idx, new_idx)}

In [ ]:
user_dict = remap(rec_train, 'user_id')
item_dict = remap(rec_train, 'app_id')

In [ ]:
rec_train['user_id'] = rec_train['user_id'].map(user_dict)
rec_train['app_id'] = rec_train['app_id'].map(item_dict)

rec_test['user_id'] = rec_test['user_id'].map(user_dict)
rec_test['app_id'] = rec_test['app_id'].map(item_dict)

In [ ]:
rec_train.groupby('user_id').apply(lambda x: display(x[['user_id', 'app_id', 'date', 'is_recommended']]))

In [ ]:
rec_train

In [ ]:
rec_test

#### Save train and test relation graph and objects mappings

In [ ]:
rec_train[['user_id', 'app_id', 'date', 'is_recommended']].to_csv("data/graph_train.csv")
rec_test[['user_id', 'app_id', 'date', 'is_recommended']].to_csv("data/graph_test.csv")

pd.DataFrame.from_dict(user_dict, orient='index').to_csv("data/user_dict.csv")
pd.DataFrame.from_dict(item_dict, orient='index').to_csv("data/item_dict.csv")

### Extract `tags` from games included in training graph by one-hot encoding them and saving indices as list

In [ ]:
from utils import *

In [ ]:
train_df = load_data_from_csv("data/graph_train.csv")
test_df = load_data_from_csv("data/graph_test.csv")

In [ ]:
item_dict = load_data_from_csv("data/item_dict.csv").to_dict()['0']

In [ ]:
meta['app_id_map'] = meta['app_id'].map(item_dict)

In [ ]:
apps_attributes = meta.dropna(subset=['app_id_map'])
apps_attributes = apps_attributes[['app_id_map', 'tags']]
apps_attributes['app_id_map'] = apps_attributes['app_id_map'].astype(int)
apps_attributes = apps_attributes.sort_values(by=['app_id_map'])

In [ ]:
apps_attributes

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
mlb = MultiLabelBinarizer()
mlb.fit_transform(apps_attributes['tags'])
tag_map = {cat: idx for idx, cat in enumerate(mlb.classes_)}

In [ ]:
apps_attributes['tags_id'] = apps_attributes['tags'].apply(lambda x: [tag_map[c] for c in x])

In [ ]:
apps_attributes = apps_attributes.explode('tags_id', ignore_index=True)

In [ ]:
apps_attributes[['app_id_map', 'tags_id']].to_csv("data/graph_app_tags.csv")

### One-hot encode app features and take app subset used in training graph

In [ ]:
games['app_id_map'] = games['app_id'].map(item_dict)
apps_attributes2 = games.dropna(subset=['app_id_map'])
apps_attributes2['app_id_map'] = apps_attributes2['app_id_map'].astype(int)
apps_attributes2 = apps_attributes2.sort_values(by=['app_id_map'])
apps_attributes2 = apps_attributes2[['app_id_map', 'win', 'mac', 'linux', 'rating', 'positive_ratio', 'user_reviews', 'price_original', 'price_final', 'discount', 'steam_deck']]

In [ ]:
def featurize_apps(df):
    for c in ['win', 'mac', 'linux', 'steam_deck']:
        df[c] = df[c].astype(int)

    rating = pd.get_dummies(df['rating'])
    df = pd.concat([df, rating], axis=1)
   
    cols = ['app_id_map', 'win', 'mac', 'linux', 'steam_deck', 'price_original', 'price_final', 'discount', 'user_reviews', 'positive_ratio'] + \
    list(rating.columns)

    return df[cols]

In [ ]:
apps_attributes2 = featurize_apps(apps_attributes2)

In [ ]:
apps_attributes2.to_csv("data/graph_app_features.csv")